<a href="https://colab.research.google.com/github/jeniferGoncalvesDaSilvaDev/gpt_from_scracth/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação de bibliotecas

In [1]:
!pip install torch
!pip install typing
!pip install pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List
from pydantic import BaseModel

class GPTInput(BaseModel):
    text: str

class GPT:
    # Mover os métodos para dentro da classe GPT com a indentação correta
    def init(self, caracteres: str):
        self.raw_text = caracteres
        self.tokens = self.tokenize(caracteres)
        self.vocab = list(set(self.tokens))
        self.token_to_idx = {ch: i for i, ch in enumerate(self.vocab)}
        self.idx_to_token = {i: ch for ch, i in self.token_to_idx.items()}
        self.encoded = self.encode(self.tokens)
        self.embedding_dim = 16
        self.embeddings = self.init_embeddings(len(self.vocab), self.embedding_dim)
        self.weights_q = self.init_weights()
        self.weights_k = self.init_weights()
        self.weights_v = self.init_weights()
        self.positional_encoding = self.get_positional_encoding(len(self.encoded), self.embedding_dim)
        self.ffn1 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)
        self.ffn2 = torch.nn.Linear(self.embedding_dim, self.embedding_dim)

    def tokenize(self, text: str) -> List[str]:
        return list(text)

    def encode(self, tokens: List[str]) -> List[int]:
        return [self.token_to_idx[tok] for tok in tokens]

    def init_embeddings(self, vocab_size: int, dim: int):
        return torch.randn(vocab_size, dim, requires_grad=True)

    def init_weights(self):
        return torch.randn(self.embedding_dim, self.embedding_dim, requires_grad=True)

    def get_positional_encoding(self, seq_len: int, dim: int):
        pe = torch.zeros(seq_len, dim)
        for pos in range(seq_len):
            for i in range(0, dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/dim)))
                if i + 1 < dim:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i)/dim)))
        return pe

    def embed(self, encoded: List[int]):
        x = torch.stack([self.embeddings[idx] for idx in encoded])
        x += self.positional_encoding
        return x

    def attention(self, x: torch.Tensor):
        Q = x @ self.weights_q
        K = x @ self.weights_k
        V = x @ self.weights_v

        dk = Q.size(-1)
        scores = (Q @ K.transpose(-2, -1)) / math.sqrt(dk)
        weights = F.softmax(scores, dim=-1)
        attended = weights @ V

        # Cálculo simbólico do número de arranjos com repetição
        n = Q.shape[0]
        num_arranjos = n ** 2
        print(f"Número de arranjos com repetição em atenção: {num_arranjos}")

        # Cálculo de combinações ponderadas entre posições Q e V
        for i in range(n):
            for j in range(n):
                prob = weights[i, j].item()
                print(f"Probabilidade de atenção do token {i} para {j}: {prob:.4f}")

        return attended

    def feed_forward(self, x):
        x = F.relu(self.ffn1(x))
        return self.ffn2(x)

    def forward(self):
        x = self.embed(self.encoded)
        x = self.attention(x)
        x = self.feed_forward(x)
        return x

    def train_step(self, target: torch.Tensor, optimizer):
        output = self.forward()
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    def validate(self):
        # Validação Bayesiana (simulação simples)
        print("Validação Bayesiana placeholder: usando heurísticas de incerteza para ajustar hiperparâmetros")
        return {
            "melhor_lr": 0.001,
            "melhor_batch_size": 8,
            "incerteza_modelo": 0.12
        }

    def retrain(self, target: torch.Tensor):
        print("Reiniciando com hiperparâmetros otimizados")
        opt = torch.optim.Adam([self.embeddings], lr=0.001)
        for i in range(10):
            loss = self.train_step(target, opt)
            print(f"Iteração {i+1} - Loss: {loss:.4f}")

# Exemplo de uso:

entrada = GPTInput(text="transformer")
# Note: The __init__ method should be defined as __init__ not init
modelo = GPT()
modelo.init(entrada.text)
target = torch.randn(len(modelo.encoded), modelo.embedding_dim)
modelo.retrain(target)
saida = modelo.forward()
print(saida)

Reiniciando com hiperparâmetros otimizados
Número de arranjos com repetição em atenção: 121
Probabilidade de atenção do token 0 para 0: 0.0000
Probabilidade de atenção do token 0 para 1: 0.0000
Probabilidade de atenção do token 0 para 2: 0.0000
Probabilidade de atenção do token 0 para 3: 0.0000
Probabilidade de atenção do token 0 para 4: 0.0000
Probabilidade de atenção do token 0 para 5: 0.0000
Probabilidade de atenção do token 0 para 6: 0.0000
Probabilidade de atenção do token 0 para 7: 0.0000
Probabilidade de atenção do token 0 para 8: 0.0000
Probabilidade de atenção do token 0 para 9: 1.0000
Probabilidade de atenção do token 0 para 10: 0.0000
Probabilidade de atenção do token 1 para 0: 0.0000
Probabilidade de atenção do token 1 para 1: 0.0000
Probabilidade de atenção do token 1 para 2: 0.0000
Probabilidade de atenção do token 1 para 3: 1.0000
Probabilidade de atenção do token 1 para 4: 0.0000
Probabilidade de atenção do token 1 para 5: 0.0000
Probabilidade de atenção do token 1 para